<a href="https://vigneashpandiyan.github.io/publications/Codes/" target="_blank" rel="noopener noreferrer">
  <img src="https://vigneashpandiyan.github.io/images/Link.png"
       style="max-width: 800px; width: 100%; height: auto;">
</a>

# Continuous Wavelet Transform (CWT) in 2D: why wavelets, what they’re used for, and why *scale* matters

## Why wavelets (and why STFT isn’t always enough)
STFT uses a **fixed window length** for the whole analysis. That means the time–frequency resolution trade-off is fixed everywhere:
- short window → good timing, poor frequency resolution
- long window → good frequency resolution, poor timing

**Wavelets** take a different approach: they use **multi‑resolution analysis**.
- At **high frequencies**, they use **short wavelets** (good time localization for fast events).
- At **low frequencies**, they use **stretched wavelets** (good frequency discrimination for slow trends).

That makes wavelets especially useful for signals with **bursts + trends** in the same record:
- impacts, crack/AE bursts, tool chatter, transients
- regime changes in process monitoring
- nonstationary vibration, biomedical signals (EEG/ECG), speech onsets
- time-localized phenomena that an “all‑time FFT” smears together

## What does CWT produce?
The CWT produces a **2D time–scale map** (often displayed as time–frequency via a scale→frequency conversion).  
It’s the wavelet cousin of a spectrogram: a **scalogram**.

## The CWT equation
A common continuous-time CWT definition is:

$$
W_x(a,b) = \frac{1}{\sqrt{|a|}} \int_{-\infty}^{\infty} x(t)\, \psi^*\!\left(\frac{t-b}{a}\right)\, dt
$$

Where:
- $x(t)$ is the signal
- $\psi(\cdot)$ is the **mother wavelet**
- $a$ is the **scale** (stretch/compress factor)
- $b$ is the **translation** (time shift)
- $^*$ denotes complex conjugate (important for complex wavelets)

**Interpretation:** slide a scaled wavelet over the signal and measure similarity.

## Scale is the main knob (like window size for STFT)
- **Small scale** $a$ → compressed wavelet → sensitive to **high frequencies**, very time-local
- **Large scale** $a$ → stretched wavelet → sensitive to **low frequencies**, less time-local

For many wavelets, an approximate scale→frequency relation is:

$$
f \approx \frac{f_c}{a\,\Delta t}
$$

- $f_c$ is the wavelet’s center frequency (depends on wavelet choice)
- $\Delta t = 1/f_s$ is the sampling period

So: **the scale range you choose decides which frequencies you can see and how “zoomed in” the analysis is.**

## Wavelet families (common examples)
You’ll see different wavelet “families” used for different goals:

- **Haar (db1)**: simplest, piecewise constant, good for edges/steps  
- **Daubechies (dbN)**: compact support, good for general-purpose DWT feature extraction  
- **Symlets (symN)**: like Daubechies but more symmetric  
- **Coiflets (coifN)**: good time–frequency localization, vanishing moments for both wavelet/scaling functions  
- **Biorthogonal (bior / rbio)**: perfect reconstruction, useful in compression/denoising  
- **Mexican hat / Ricker**: real-valued, good for peaks/impulses  
- **Morlet**: complex, sinusoid in a Gaussian → great for oscillatory content (**we use this here**)  
- **DOG/Paul**: other CWT families often used in physics / geoscience

In this notebook we use a **Morlet-type wavelet** (PyWavelets name: `morl`) because it’s intuitive: it behaves like a short “wave packet” that locks onto oscillations.


## Demo: Morlet CWT scalogram and the effect of scale range

We use the same style as the STFT lecture demo:
- A 100 Hz tone early
- A 350 Hz tone mid
- A chirp (200→700 Hz) late

Then we compute **three scalograms** using different **scale ranges** (small / medium / large).  
Each plot is a separate cell (separate call), so it’s slide-friendly.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import chirp

# PyWavelets for CWT
try:
    import pywt
except ImportError as e:
    raise ImportError(
        "PyWavelets is required for this notebook. Install with: %pip install pywavelets"
    ) from e

# -----------------------------
# Signal (same idea as STFT demo)
# -----------------------------
fs = 2000  # Hz
T = 2.0
t = np.arange(int(T * fs)) / fs

x1 = (t < 0.8) * np.sin(2*np.pi*100*t)
x2 = ((t >= 0.8) & (t < 1.3)) * 0.9*np.sin(2*np.pi*350*t)
x3 = (t >= 1.3) * 0.8*chirp(t, f0=200, f1=700, t1=T, method="linear")

rng = np.random.default_rng(0)
x = x1 + x2 + x3 + 0.05*rng.standard_normal(len(t))

wavelet = "morl"
dt = 1/fs

def cwt_to_freq_hz(wavelet_name: str, scales: np.ndarray, dt: float) -> np.ndarray:
    # PyWavelets uses: frequency = scale2frequency(wavelet, scale) / dt
    return pywt.scale2frequency(wavelet_name, scales) / dt


### How to read these plots
- The vertical axis is shown in **Hz** by converting scale→frequency.
- **Small scales** correspond to **higher frequencies**.
- **Larger scale ranges** extend the plot toward **lower frequencies**, but can blur timing.

The dashed vertical lines mark the event boundaries at **0.8 s** and **1.3 s**.


In [ ]:
# ============================================================
# PLOT 1: SMALL scales (high-frequency focus)
# Expect: great localization for fast/high-frequency content, limited low-frequency coverage.
# ============================================================
scales_small = np.arange(1, 64)  # small scales -> higher frequencies
coef, freqs_hz = pywt.cwt(x.astype(float), scales_small, wavelet, sampling_period=dt)

power = np.abs(coef)
plt.figure(figsize=(9, 4.5), dpi=150)
plt.imshow(
    power,
    aspect="auto",
    origin="lower",
    extent=[t[0], t[-1], freqs_hz.min(), freqs_hz.max()],
)
plt.title("CWT Scalogram (Morlet) — SMALL scales: 1–63 (high-frequency emphasis)")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.axvline(0.8, linestyle="--", linewidth=1)
plt.axvline(1.3, linestyle="--", linewidth=1)
cbar = plt.colorbar()
cbar.set_label("|CWT coef| (a.u.)")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# PLOT 2: MEDIUM scales (balanced view)
# Expect: broader frequency coverage while keeping decent timing.
# ============================================================
scales_med = np.arange(1, 128)  # medium scale range
coef, freqs_hz = pywt.cwt(x.astype(float), scales_med, wavelet, sampling_period=dt)

power = np.abs(coef)
plt.figure(figsize=(9, 4.5), dpi=150)
plt.imshow(
    power,
    aspect="auto",
    origin="lower",
    extent=[t[0], t[-1], freqs_hz.min(), freqs_hz.max()],
)
plt.title("CWT Scalogram (Morlet) — MEDIUM scales: 1–127 (balanced)")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.axvline(0.8, linestyle="--", linewidth=1)
plt.axvline(1.3, linestyle="--", linewidth=1)
cbar = plt.colorbar()
cbar.set_label("|CWT coef| (a.u.)")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# PLOT 3: LARGE scales (adds low-frequency detail)
# Expect: stronger low-frequency visibility; time-localization is less sharp at low freq.
# ============================================================
scales_large = np.arange(1, 256)  # large scale range -> includes lower frequencies
coef, freqs_hz = pywt.cwt(x.astype(float), scales_large, wavelet, sampling_period=dt)

power = np.abs(coef)
plt.figure(figsize=(9, 4.5), dpi=150)
plt.imshow(
    power,
    aspect="auto",
    origin="lower",
    extent=[t[0], t[-1], freqs_hz.min(), freqs_hz.max()],
)
plt.title("CWT Scalogram (Morlet) — LARGE scales: 1–255 (more low-frequency coverage)")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.axvline(0.8, linestyle="--", linewidth=1)
plt.axvline(1.3, linestyle="--", linewidth=1)
cbar = plt.colorbar()
cbar.set_label("|CWT coef| (a.u.)")
plt.tight_layout()
plt.show()


## Key takeaway (the “window size” equivalent)
In STFT, you pick a **window length**. In CWT, you pick a **scale range**.

- If you choose only small scales, you’ll miss low-frequency structure.
- If you include very large scales, you’ll see low-frequency content—but those components inherently have broader time support.

So scale selection is not decoration; it’s the **lens** you’re analyzing with.
